In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime, date_format, col, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, ArrayType, DoubleType
import requests
import logging
from datetime import datetime

In [0]:
# Configuracao de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Iniciando Spark
spark = SparkSession.builder.appName("Clima-Api").getOrCreate()

#pegando a data para colocar no nome do arquivo
data_atual = datetime.now().strftime("%Y%m%d_%H%M%S")

# Configuracao da API
api_key = "eac87a91b5796009567e68258a4f3f8c"
cidade = "Sao Paulo,BR"
url = f"https://api.openweathermap.org/data/2.5/weather?q={cidade}&appid={api_key}&units=metric&lang=pt_br"

# Caminhos para salvar
caminho_bronze = "/Volumes/projeto/default/bronze"
caminho_erro = "/Volumes/projeto/default/erros/climaerro.parquet"
caminho_bronze_clima_data = f"{caminho_bronze}/clima_{data_atual}.parquet"
# Definindo o schema
clima_schema = StructType([
    StructField("name", StringType(), True),
    StructField("weather", ArrayType(
        StructType([
            StructField("description", StringType(), True)
        ])
    ), True),
    StructField("main", StructType([
        StructField("temp", FloatType(), True),
        StructField("feels_like", DoubleType(), True),
        StructField("temp_min", FloatType(), True),
        StructField("temp_max", FloatType(), True),
        StructField("humidity", IntegerType(), True)
    ]), True),
    StructField("dt", IntegerType(), True)
])


try:
    # Requisitando os dados da API
    response = requests.get(url, timeout=10)
    response.raise_for_status()

    dados_clima = response.json()

    # Validação: verifica se veio o campo 'main'
    main_info = dados_clima.get('main', {})
    if not main_info:
        raise ValueError("Nenhum dado de clima encontrado no campo 'main'.")

    # Cria DataFrame diretamente
    df_clima = spark.createDataFrame([dados_clima], schema=clima_schema)

    # Tenta salvar
    try:
        df_clima.write.mode("overwrite").parquet(caminho_bronze_clima_data)
        logger.info("Dados de clima salvos com sucesso em Parquet.")
        df_clima.printSchema()
        df_clima.show()
    except Exception as e:
        raise ValueError(f"Erro ao salvar os dados de clima em Parquet: {e}")

except requests.exceptions.RequestException as e:
    logger.error(f"Erro ao fazer requisição para a API: {e}")
    df_erro = spark.createDataFrame([], clima_schema).withColumn("erro_carregamento", lit(str(e)))
    df_erro.write.mode("overwrite").parquet(caminho_erro)

except ValueError as e:
    logger.error(f"Erro de valor: {e}")
    df_erro = spark.createDataFrame([], clima_schema).withColumn("erro_carregamento", lit(str(e)))
    df_erro.write.mode("overwrite").parquet(caminho_erro)

except Exception as e:
    logger.error(f"Erro desconhecido: {e}")
    df_erro = spark.createDataFrame([], clima_schema).withColumn("erro_carregamento", lit(str(e)))
    df_erro.write.mode("overwrite").parquet(caminho_erro)
